# Before proceeding, go to "File" and select option "Save a copy in drive" then and only then proceed. <br>
# <font color="red">  YOU WILL NOT BE ABLE TO SAVE CHANGES IN THIS NOTEBOOK </font>

In [ ]:
!pip install langchain chromadb pypdf sentence_transformers

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from pypdf import PdfReader
import requests
from google.colab import userdata
hf_bearer = userdata.get('HuggingFaceBearer')
API_URL = "https://api-inference.huggingface.co/models/google/gemma-1.1-7b-it"
headers = {"Authorization": hf_bearer}

In [ ]:
reader = PdfReader("example.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(pdf_texts[0])

Alphabet Announces Fourth Quarter and Fiscal Year  2023  Results
MOUNTAIN VIEW, Calif. – January 30, 2024  – Alphabet Inc. (NASDAQ: GOOG, GOOGL) today announced 
financial results for the quarter and fiscal year ended  December 31, 2023 .
Sundar Pichai, CEO, said: “We are pleased with the ongoing strength in Search and the growing contribution from 
YouTube and Cloud. Each of these is already benefiting from our AI investments and innovation. As we enter the 
Gemini era, the best is yet to come.”
Ruth Porat, President and Chief Investment Officer; CFO said: “We ended 2023 with very strong fourth quarter 
financial results, with Q4 consolidated revenues of $86 billion, up 13% year over year. We remain committed to our 
work to durably re-engineer our cost base as we invest to support our growth opportunities.”
Q4 2023  Financial Highlights
The following table summarizes our consolidated financial results for the quarters and years ended December 31, 
2022  and 2023  (in millions, except

In [ ]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(character_split_texts[10])
print(f"\nTotal chunks: {len(character_split_texts)}")

supplemental information regarding our performance and liquidity by excluding certain items that may not be 
indicative of our recurring core business operating results, such as our revenues excluding the effect of foreign 
exchange rate movements and hedging activities, which are recognized at the consolidated level. We believe that 
both management and investors benefit from referring to these non-GAAP financial measures in assessing our 
performance and when planning, forecasting, and analyzing future periods. These non-GAAP financial measures 
also facilitate management’s internal comparisons to our historical performance and liquidity as well as comparisons 
to our competitors’ operating results. We believe these non-GAAP financial measures are useful to investors both 
because (1) they allow for greater transparency with respect to key metrics used by management in its financial and

Total chunks: 32


In [ ]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(token_split_texts[10])
print(f"\nTotal chunks: {len(token_split_texts)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


supplemental information regarding our performance and liquidity by excluding certain items that may not be indicative of our recurring core business operating results, such as our revenues excluding the effect of foreign exchange rate movements and hedging activities, which are recognized at the consolidated level. we believe that both management and investors benefit from referring to these non - gaap financial measures in assessing our performance and when planning, forecasting, and analyzing future periods. these non - gaap financial measures also facilitate management ’ s internal comparisons to our historical performance and liquidity as well as comparisons to our competitors ’ operating results. we believe these non - gaap financial measures are useful to investors both because ( 1 ) they allow for greater transparency with respect to key metrics used by management in its financial and

Total chunks: 40


In [ ]:
embedding_function = SentenceTransformerEmbeddingFunction()

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("alphabet_annual_report", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

40

In [ ]:
def rag(query, retrieved_documents):
    information = "\n\n".join(retrieved_documents)

    messages = {"inputs": f"""Answer solely based on Question and informatino provided. Question: {query}. \n Information: {information}"""}

    response = requests.post(API_URL, headers=headers, json=messages)
    return response.json()

In [ ]:
query = "What are cloud services"

results = chroma_collection.query(query_texts=[query], n_results=1)
retrieved_documents = results['documents'][0]

output = rag(query=query, retrieved_documents=retrieved_documents)
print(output[0]["generated_text"])

Answer solely based on Question and informatino provided. Question: What are cloud services. 
 Information: • google cloud includes infrastructure and platform services, collaboration tools, and other services for enterprise customers. google cloud generates revenues primarily from consumption - based fees and subscriptions received for google cloud platform services, google workspace communication and collaboration tools, and other enterprise services. • other bets is a combination of multiple operating segments that are not individually material. revenues from other bets are generated primarily from the sale of healthcare - related services and internet services. certain costs are not allocated to our segments because they represent alphabet - level activities. these costs primarily include ai - focused shared r & d activities, including development costs of our general ai models ; corporate initiatives such as our philanthropic activities ; corporate shared costs such as certain fin

<div align="center">

  Thank you for joining us Gemma Day Extended, hope to see you next soon!
</div>

<p align="center">
  <img src="https://i.pinimg.com/originals/2b/a7/17/2ba717ac010d3b9f8d26e9db4da291b8.jpg" alt="X.com/akajammythakkar" width="20" height="20">
  <img src="https://w7.pngwing.com/pngs/521/488/png-transparent-logo-computer-icons-instagram-logo-miscellaneous-text-logo.png" alt="instagram.com/akajammythakkar" width="20" height="20">
  <img src="https://w7.pngwing.com/pngs/490/260/png-transparent-email-email-thumbnail.png" alt="akajammythakkar@gmail.com" width="20" height="20">
  <img src="https://w7.pngwing.com/pngs/940/589/png-transparent-linkedin-free-text-telephone-call-trademark-thumbnail.png" alt="linkedin.com/in/akajammythakkar" width="20" height="20">
  <a href="https://imjt.dev" target="_blank">@akajammythakkar</a>
</p>
